In [186]:
!pip install transformers
from transformers import (
    GPT2Tokenizer,
    DataCollatorForLanguageModeling,
    TextDataset,
    GPT2LMHeadModel,
    TrainingArguments,
    Trainer,
    pipeline)

## Описание датасета

Этот набор данных содержит 149113 неприемлемых комментариев, то есть сообщений на деликатную тему, которые могут расстроить читателя или нанести ущерб репутации говорящего. Понятие неуместности довольно близко к токсичности, однако сама явная токсичность, а также явная непристойность были намеренно исключены из набора данных.

Источники данных: комментарии к постам на 2ch и pikabu

### Загрузим данные

In [187]:
from google.colab import drive

drive.mount('/content/drive')
%cd 'drive/MyDrive/NLP'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive/NLP'
/content/drive/MyDrive/NLP


In [188]:
train_data = 'Data_lab2/train.txt'
val_data = 'Data_lab2/val.txt'

При использовании предварительно обученной модели трансформеров необходимо использовать соответствующий, предварительно обученный, токенизатор:

In [4]:
tokenizer = GPT2Tokenizer.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')

Downloading:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

[DataCollator](https://github.com/huggingface/transformers/blob/master/src/transformers/data/data_collator.py) - это функция, используемая для формирования батча из обучающего и тестового набора данных.

In [5]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Мы будем использовать HuggingFace реализацию [TextDataset](https://github.com/huggingface/transformers/blob/master/src/transformers/data/datasets/language_modeling.py). TextDataset разбивает текст на последовательные блоки определенной длины, например, вырезаем текст каждые 128 токенов.

In [6]:
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=train_data,
    block_size=128)
     
test_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=val_data,
    block_size=128)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [7]:
print(tokenizer.decode(train_dataset[7]))

кий. Но ток не надо сравнивать офицера с исправившимся уголовником. У нас в стране канешн все равны )) но только не говорите об этом прокурорам и депутатам, засмеют
Меня провайдер не пускает, утверждая, что это находится в списке экстремистских материалов :D А вот и он, кстати: url
Там мало того, что бюджет колоссальный на армию, так еще и призывное рабство до сих пор поддерживают. 
Не знаю. Сколько? (Хорониться не планирую, предпочитаю стать пеплом) 
Ей будет просто сделать карьеру в порноиндустрии.
Cуи


Мы будем использовать предварительно обученную модель [sberbank-ai/rugpt3small_based_on_gpt2](https://huggingface.co/sberbank-ai/rugpt3small_based_on_gpt2), загрузив ее с помощью метода `from_pretrained()`.

In [8]:
model = GPT2LMHeadModel.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')

Downloading:   0%|          | 0.00/526M [00:00<?, ?B/s]

Класс [Trainer](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Trainer) предоставляет интерфейс для полнофункционального обучения - он позволяет обучать, настраивать и оценивать любую модель трасформеров. Он принимает в качестве входных данных: модель, обучающие аргументы, набор данных, DataCollator, токенизатор и т.д.

[TrainingArguments](https://huggingface.co/docs/transformers/main_classes/trainer#trainingarguments) - это подмножество аргументов, которые относятся к циклу обучения - мы можем настроить, например: batch size, learning rate, количество эпох.

In [9]:
training_args = TrainingArguments(
    output_dir = 'Data_lab2/out', 
    overwrite_output_dir = True, 
    per_device_train_batch_size = 16, 
    per_device_eval_batch_size = 16, 
    learning_rate = 5e-5, 
    num_train_epochs = 3, 
)

trainer = Trainer(
    model = model,
    args = training_args,
    data_collator=data_collator,
    train_dataset = train_dataset,
    eval_dataset = test_dataset
)

In [10]:
trainer.train()

***** Running training *****
  Num examples = 26849
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 5037


Step,Training Loss
500,4.436700
1000,4.382900
1500,4.372000
2000,4.230500
2500,4.160800
3000,4.163700
3500,4.122200
4000,4.031100
4500,4.032200
5000,4.032900


Saving model checkpoint to Data_lab2/out/checkpoint-500
Configuration saved in Data_lab2/out/checkpoint-500/config.json
Model weights saved in Data_lab2/out/checkpoint-500/pytorch_model.bin
Saving model checkpoint to Data_lab2/out/checkpoint-1000
Configuration saved in Data_lab2/out/checkpoint-1000/config.json
Model weights saved in Data_lab2/out/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to Data_lab2/out/checkpoint-1500
Configuration saved in Data_lab2/out/checkpoint-1500/config.json
Model weights saved in Data_lab2/out/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to Data_lab2/out/checkpoint-2000
Configuration saved in Data_lab2/out/checkpoint-2000/config.json
Model weights saved in Data_lab2/out/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to Data_lab2/out/checkpoint-2500
Configuration saved in Data_lab2/out/checkpoint-2500/config.json
Model weights saved in Data_lab2/out/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to Data_lab2/o

TrainOutput(global_step=5037, training_loss=4.195000602111755, metrics={'train_runtime': 6924.352, 'train_samples_per_second': 11.632, 'train_steps_per_second': 0.727, 'total_flos': 5261572325376000.0, 'train_loss': 4.195000602111755, 'epoch': 3.0})

In [11]:
trainer.save_model()

Saving model checkpoint to Data_lab2/out
Configuration saved in Data_lab2/out/config.json
Model weights saved in Data_lab2/out/pytorch_model.bin


Чтобы сгенерировать текст, используем [pipeline](https://huggingface.co/docs/transformers/main_classes/pipelines#transformers.TextGenerationPipeline) для генерации текста:

In [12]:
generator = pipeline('text-generation', tokenizer='sberbank-ai/rugpt3small_based_on_gpt2', model='Data_lab2/out')

loading configuration file Data_lab2/out/config.json
Model config GPT2Config {
  "_name_or_path": "Data_lab2/out",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 2048,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 2048,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "torch_dtype": "float32",
  "transformers_version": "4.14.1",
  "use_cache": true,
  "vocab_size": 50264
}

loading configuration file Data_lab2/out/config.json
Model config GPT2Config {
  "_

Сгенерируем несколько смешных/кринжовых текстов:

In [147]:
text_random_sampling = generator('Наша работа и опасна, и трудна',
                                 max_length=40,
                                 top_k=0,
                                 do_sample=True,
                                 temperature=1.33)
print(text_random_sampling[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Наша работа и опасна, и трудна для здоровья, нельзя ни трусы надеть, ни пиджак одеть чтобы выполнение задач развивала чувство свободы. это как ритуал на свадьбу: выбираешь религию вождя


In [152]:
text_random_sampling = generator('уехал в изумрудный лес, едем на помощь',
                                 max_length=40,
                                 top_k=0,
                                 do_sample=True,
                                 temperature=0.99)
print(text_random_sampling[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


уехал в изумрудный лес, едем на помощь двум лесбиянкам, латексная пленка заканчивается, одна начинает исполнять пуританские ритуалы (с).
Да назовут


In [178]:
text_random_sampling = generator('каждый десятый житель России',
                                 max_length=40,
                                 top_k=0,
                                 do_sample=True,
                                 temperature=0.87)
print(text_random_sampling[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


каждый десятый житель России чувствует себя очень плохо, но в то же время в России на удивление хорошо.
Можно бы было все животы укоротить и носить круглые шортики
После


In [185]:
text_random_sampling = generator('каждый второй житель России',
                                 max_length=40,
                                 top_k=0,
                                 do_sample=True,
                                 temperature=0.77)
print(text_random_sampling[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


каждый второй житель России (которые не просто так едут в Америку) не знает, что он гомосексуалист. Если бы у нас не было возможности жениться на чернокожих, я бы не стеснялся.


In [217]:
text_random_sampling = generator('заходит улитка в бар и спрашивает',
                                 max_length=40,
                                 top_k=0,
                                 do_sample=True,
                                 temperature=0.89)
print(text_random_sampling[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


заходит улитка в бар и спрашивает у меня сумку. Приходит через пять минут - кто забыл?
конечно ересь, но все же...😂😂😂😂�


In [339]:
text_random_sampling = generator('В будущем все люди',
                                 max_length=27,
                                 top_k=0,
                                 do_sample=True,
                                 temperature=0.79)
print(text_random_sampling[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


В будущем все люди будут жить в одном доме, а потом прочтут пост, как они живут в одном доме.
Я был на


По итогу имеем несколько приемлемых попыток:
 - Наша работа и опасна, и трудна для здоровья, нельзя ни трусы надеть, ни пиджак одеть чтобы выполнение задач развивала чувство свободы. это как ритуал на свадьбу: выбираешь религию вождя
 - уехал в изумрудный лес, едем на помощь двум лесбиянкам, латексная пленка заканчивается, одна начинает исполнять пуританские ритуалы (с).
 - каждый десятый житель России чувствует себя очень плохо, но в то же время в России на удивление хорошо. Можно бы было все животы укоротить и носить круглые шортики
 - каждый второй житель России (которые не просто так едут в Америку) не знает, что он гомосексуалист. Если бы у нас не было возможности жениться на чернокожих, я бы не стеснялся.
 - заходит улитка в бар и спрашивает у меня сумку. Приходит через пять минут - кто забыл? конечно ересь, но все же...😂😂😂😂�
 - В будущем все люди будут жить в одном доме, а потом прочтут пост, как они живут в одном доме.